In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Imports

In [3]:
import os
import pandas as pd
import tqdm
import re
import torch
import json
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf
from preprocess import *
from vocabulary import *
from model import *
from model_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\swaga\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Pre Processing Dataset

In [5]:
# train_df = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_train.csv')
# val_df   = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_dev.csv')
# # train_df = pd.read_csv('../datasets/snli/snli_/snli_1.0_train.csv')
# # val_df   = pd.read_csv('../datasets/snli/snli_/snli_1.0_dev.csv')
# train_df, val_df = preprocess_text(train_df, val_df)
# print(len(train_df))
# print(len(val_df))

In [8]:
# len(train_df['sentence2'][8])

In [9]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
rw = pd.read_csv('test_sets/ins.csv', encoding='ansi')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw.drop(['original_summary'], axis = 1, inplace=True)
rw.rename(columns = {'text':'sentence1'}, inplace = True)
rw.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)

# print(rw)
gold = [-1] * 100
rw.insert(loc=0, column='gold_label', value=gold)
rw = rw[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw = rw.dropna()
rw.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appointed sarah al ceo in...,saudi stock exchange appointed sarah al ceo i...
1,-1,hollywood actor morgan freeman said would like...,hollywood actor morgan freeman said would lik...
2,-1,uk india business council chairperson said 60 ...,india business council chairperson said 60 br...
3,-1,indian railways wednesday increased lower bert...,indian railways wednesday increased lower ber...
4,-1,two pilots arrested saturday glasgow scotland ...,two pilots arrested saturday glasgow scotland...


In [10]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
rw_ = pd.read_csv('test_sets/ins.csv', encoding='ansi')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')

rw_.drop(['text'], axis = 1, inplace=True)
rw_.rename(columns = {'original_summary':'sentence1'}, inplace = True)
rw_.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)

gold = [-1] * 100
rw_.insert(loc=0, column='gold_label', value=gold)
rw_ = rw_[['gold_label', 'sentence1', 'sentence2']]
# rw = rw[['sentence1', 'sentence2']]
rw_ = rw_.dropna()
rw_.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appoints its first female...,saudi stock exchange appointed sarah al ceo i...
1,-1,what makes fireworks explode,hollywood actor morgan freeman said would lik...
2,-1,60 british firms to increase investment in india,india business council chairperson said 60 br...
3,-1,railways increase senior citizen women quota,indian railways wednesday increased lower ber...
4,-1,pilots arrested in scotland for drinking befor...,two pilots arrested saturday glasgow scotland...


In [11]:
rw_.head()

,gold_label,sentence1,sentence2
0,-1,saudi stock exchange appoints its first female...,saudi stock exchange appointed sarah al ceo i...
1,-1,what makes fireworks explode,hollywood actor morgan freeman said would lik...
2,-1,60 british firms to increase investment in india,india business council chairperson said 60 br...
3,-1,railways increase senior citizen women quota,indian railways wednesday increased lower ber...
4,-1,pilots arrested in scotland for drinking befor...,two pilots arrested saturday glasgow scotland...


In [12]:
rw['sentence1'][0]

'saudi stock exchange appointed sarah al ceo investment bank ncb capital first female head first woman chair major government financial institution kingdom chair arab world 39 largest stock exchange time preparing offer shares public '

In [13]:
rw_['sentence2'][0]

' saudi stock exchange appointed sarah al ceo investment bank ncb capital first female'

In [14]:
len(rw['sentence2'][8])

117

In [15]:
# train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
# train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
# val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
# val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw['sentence1'] = rw['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw['sentence2'] = rw['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw_['sentence1'] = rw_['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw_['sentence2'] = rw_['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [16]:
# train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
# val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
# print(train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)])
# print(val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)])

In [17]:
# train_df.head()

In [18]:
# val_df.head()

In [19]:
# train_val_df = pd.concat([train_df, val_df])

In [21]:
# sentence_pairs, _ = pair_generator(train_val_df)
rw_sentence_pairs, __ = pair_generator(rw)
rw_sentence_pairs_, __ = pair_generator(rw_)
# train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
# val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

# labels = set(train_sentence_labels)
# print(labels)

# tag2idx = {word: i for i, word in enumerate(labels)}
# print(tag2idx)
tag2idx = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
print(tag2idx)

# train_labels = [tag2idx[t] for t in train_sentence_labels]
# val_labels = [tag2idx[t] for t in val_sentence_labels]

{'entailment': 0, 'neutral': 1, 'contradiction': 2}


In [22]:
rw_sentence_pairs_[0]

('saudi stock exchange appoints its first female head',
 'saudi stock exchange appointed sarah al ceo investment bank ncb capital first female')

In [23]:
# vocab = Vocabulary()

# for data in [rw_sentence_pairs]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)
    
# for data in [rw_sentence_pairs_]:
#   for sen in data:
#     premise    = sen[0]
#     hypothesis = sen[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# for data in [sentence_pairs]:
#   for sentence_pair in data:
#     premise    = sentence_pair[0]
#     hypothesis = sentence_pair[1]
#     vocab.addSentence(premise)
#     vocab.addSentence(hypothesis)

# with open('saved_vocabs/voc.class', 'wb') as vocab_file:
#     pickle.dump(vocab, vocab_file)
# print("Vocab size:", len(vocab.word2index))

In [28]:
with open('saved_vocabs/voc.class', 'rb') as vocab_file_r:
    vocab = pickle.load(vocab_file_r)
print("Vocab size:", len(vocab.word2index))

Vocab size: 33830


In [29]:
index2word = {}
for wrd, idx in vocab.word2index.items():
    # print(wrd, idx)
    index2word[idx] = wrd

## Model

In [ ]:
EPOCHS = 15
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 128
LEARNING_RATE = 0.005
STACKED_LAYERS = 2
EMBEDDING_PATH = '../../../../embeddings/google_news/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '../../../../embeddings/glove/glove.6B.300d.txt'
# EMBEDDING_PATH = '../embeddings/google_news/GoogleNews-vectors-negative300.bin'
# GLOVE_EMBEDDING = '../embeddings/glove/glove.6B.300d.txt'

initiate_model_vocab(vocab, tag2idx)

In [ ]:
train_data = DataSetLoader(get_pair_indices(vocab, train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(vocab, val_sentence_pairs), val_labels)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

In [ ]:
embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [ ]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [ ]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in lstm_model.state_dict():
    print(param_tensor, "\t", lstm_model.state_dict()[param_tensor].size())

In [ ]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(lstm_model, 'sm_2')

In [30]:
lstm_model = torch.load('saved_models/sm_2')
lstm_model.eval()

LSTM(
  (embedding): Embedding(33589, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=512, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [31]:
print(lstm_model.output)

Linear(in_features=32, out_features=3, bias=True)


In [ ]:
sentence_o = [''] * len(rw)
sentence_p = [''] * len(rw)
# sentence_o = ['']
# sentence_p = ['']

# sentence_o[0] = "rock music is very good for public"
# sentence_p[0] = "people dont like rock and roll"
# sentence_o = "uk india business council chairperson said 60 british firms likely increase investment india next years firms positive reforms fdi introduced last two years added india even important britain said"
# sentence_p = "60 british firms to increase investment in india"
# sentence_o = clean_text(sentence_o)
# sentence_p = clean_text(sentence_p)


# sen_p1 = [(sentence_o, sentence_p)]
# # sen_p2 = []
# for i in range(rw):
#   sen1 = sentence_o[i]
#   sen2 = sentence_p[i]
#   sen_p1.append((sen1, sen2))
# for i in range(rw_):
#   sen2 = sentence_o[i]
#   sen1 = sentence_p[i]
#   sen_p2.append((sen1, sen2))

# sen_p1

In [32]:
rrw_sentence_pairs_ = []
for i in range(len(rw_sentence_pairs_)):
    rrw_sentence_pairs_.append((rw_sentence_pairs_[i][1], rw_sentence_pairs_[i][0]))
print('rw[0]: ', rw_sentence_pairs_[0])
print('rrw[0]: ', rrw_sentence_pairs_[0])

rw[0]:  ('saudi stock exchange appoints its first female head', 'saudi stock exchange appointed sarah al ceo investment bank ncb capital first female')
rrw[0]:  ('saudi stock exchange appointed sarah al ceo investment bank ncb capital first female', 'saudi stock exchange appoints its first female head')


In [33]:
# rw_sentence_pairs, __ = pair_generator(rw)

In [34]:
id_pairs = get_pair_indices(vocab, rw_sentence_pairs)
id_pairs_1 = get_pair_indices(vocab, rw_sentence_pairs_)
id_pairs_2 = get_pair_indices(vocab, rrw_sentence_pairs_)
# id_pairs = get_pair_indices(vocab, sen_p1)
# id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [ ]:
id_pairs

In [35]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [36]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_1]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_1]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_1 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_1.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [37]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs_2]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs_2]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction_2 = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction_2.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [ ]:
prediction

In [38]:
soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
soft.tolist()
soft_1 = torch.log_softmax(prediction_1, dim=1).argmax(dim=1)
soft_1.tolist()
soft_2 = torch.log_softmax(prediction_2, dim=1).argmax(dim=1)
soft_2.tolist()

[2,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 0,
 2,
 0,
 1]

In [39]:
print(soft, soft_1, soft_2)

tensor([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0') tensor([1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 2,
        2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1,
        1, 2, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1,
        1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1,
        2, 2, 1, 2], device='cuda:0') tensor([2, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1,
        1, 1, 1, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 0, 0, 2, 1,
        2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 2, 1, 0, 1, 2, 2, 1, 0, 0, 0, 1, 0, 2,
        0, 1, 1, 0, 0, 2, 2, 0, 2, 0, 2, 0, 

In [42]:
ctr = 0
for i in range(len(soft)):
    if soft[i] == 2:
        print(rw_sentence_pairs[i])
        ctr = ctr + 1
print(ctr)

('bjp led opposition friday slammed nitish kumar led state government bihar assembly alleged increase crimes state including recent killing opposition leaders opposition also brought alleged rape minor girl rjd mla raised anti government slogans following houses adjourned', 'opposition also brought alleged rape minor girl rjd mla raised anti government slogans following')
('chinese president xi jinping told ukrainian counterpart petro poroshenko china willing play constructive role seeking political resolution ukraine crisis also discussed opportunities economic development bilateral meeting sidelines world economic forum eastern ukraine facing civil war since russia annexed crimea', 'chinese president xi jinping told ukrainian counterpart petro por')
('part cricketer ravindra jadeja restaurant jaddu food field rajkot demolished rajkot municipal corporation drive illegal construction city thursday store room kitchen constructed illegally part restaurant pulled corporation earlier issue

In [ ]:
ctr_1 = 0
for i in range(len(soft_1)):
    if soft_1[i] == 2:
        print(rw_sentence_pairs_[i])
        ctr_1 = ctr_1 + 1
print(ctr_1)

In [ ]:
ctr_2 = 0
for i in range(len(soft_2)):
    if soft_2[i] == 0:
        print(rrw_sentence_pairs_[i])
        ctr_2 = ctr_2 + 1
print(ctr_2)

In [ ]:
# id_pairs = get_pair_indices(vocab, rrw_sentence_pairs)
# # id_pairs = get_pair_indices(vocab, sen_p2)
# id_pairs[0]

In [ ]:
# premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
# hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

# premise_len    = list(map(len, premise_seq))
# hypothesis_len = list(map(len, hypothesis_seq))

# batch = len(premise_seq)
# temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
# premise_seq    = temp[:batch, :]
# hypothesis_seq = temp[batch:, :]

# prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# # prediction = prediction[prediction!=prediction[0,3]]

# prediction_list = prediction.to('cpu')
# prediction_list = prediction_list.tolist()
# print(len(prediction_list))

In [ ]:
# prediction

In [ ]:
# soft2 = torch.log_softmax(prediction, dim=1).argmax(dim=1)
# soft2.tolist()

In [ ]:
# ctr2 = 0
# for i in range(len(soft)):
#     if soft2[i] == 0:
#         print(rrw_sentence_pairs[i])
#         ctr2 = ctr2 + 1
# print(ctr2)

In [43]:
e_100 = 0   # perfectly entailed pairs
e_50 = 0    # entailment
n_100 = 0   # perfectly neutral pairs
n_50 = 0    # neutral
cc = 0      # contradicting pairs
for i in range(len(soft_1)):
    if soft_1[i] == soft_2[i] == 0:
        e_100 = e_100 + 1
    elif (soft_1[i] == 0 and soft_2[i] == 1) or (soft_1[i] == 1 and soft_2[i] == 0) or (soft_1[i] == 0 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 0):
        e_50 = e_50 + 1
    elif soft_1[i] == soft_2[i] == 1:
        n_100 = n_100 + 1
    elif (soft_1[i] == 1 and soft_2[i] == 2) or (soft_1[i] == 2 and soft_2[i] == 1):
        n_50 = n_50 + 1
    else: 
        cc = cc + 1
print('e100: ', e_100)
print('e50: ', e_50)
print('n100: ', n_100)
print('n50: ', n_50)
print('c100: ', cc)

e100:  1
e50:  38
n100:  19
n50:  31
c100:  11


In [ ]:
rw_sentence_pairs[0]

In [ ]:
prediction[0]

In [ ]:
rw_sentence_pairs[1]
# 0: entailment
# 1: neutral
# 2: contradiction

In [44]:
maxi = torch.max(prediction, dim=0)
mini = torch.min(prediction, dim=0)
# print(maxi[0][0])
denom1 = float(maxi[0][0].to('cpu')) - float(mini[0][0].to('cpu'))
denom2 = float(maxi[0][1].to('cpu')) - float(mini[0][1].to('cpu'))
denom3 = float(maxi[0][2].to('cpu')) - float(mini[0][2].to('cpu'))
denom = ( denom1 + denom2 + denom3 ) / 3
print(maxi, mini)
print(denom)

torch.return_types.max(
values=tensor([1.4908, 1.2091, 0.6716], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([ 4, 18, 96], device='cuda:0')) torch.return_types.min(
values=tensor([-0.4321, -0.3117, -1.3790], device='cuda:0', grad_fn=<MinBackward0>),
indices=tensor([18, 96, 79], device='cuda:0'))
1.8314221799373627


In [45]:
aa = []
for i in prediction:
    a = []
    for j in i:
        a.append(float(j.to('cpu'))/denom)
    aa.append(a)
print(aa)

[[0.24731164388214988, 0.2060030725278753, -0.3668391358498671], [0.2576244015869797, -0.0989217967988879, 0.22527954228457175], [0.6017685480474289, -0.023536088484584904, -0.39276739559317936], [0.3295480524571151, -0.062226016292205415, -0.05347200336468929], [0.8139928166435216, -0.1578474866966397, -0.16294091431617133], [0.2888357481508716, 0.29973980970713704, -0.5425112352532242], [0.23695286676757632, 0.18005984183089815, -0.32189193929844584], [0.18611521831450137, 0.2869075544099377, -0.34782635015098123], [0.5608006635256847, -0.09983373942316362, -0.1724037286722289], [0.2663135756396287, -0.04403026632138657, -0.001951516696738485], [0.20689257499442942, 0.10309763514096805, -0.17841163746904898], [0.24805892228888762, 0.22811194908131313, -0.3938979629592895], [0.3642293731705637, -0.08422272867078061, -0.013638229605968893], [0.25862221564874405, 0.1918627626997583, -0.3690941520184157], [0.42002214692230305, 0.23649237976017543, -0.6698770791949291], [0.342864650650924

In [46]:
e_scores = []
for i in aa:
    e_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(e_scores)

[0.28686092639838184, -0.006846415638814407, 0.3931071597593263, 0.14076281907028068, 0.35920399504454925, 0.3924867553030657, 0.2595498907925885, 0.2709581382681806, 0.2733150089945327, 0.09849998367026956, 0.17474121201358497, 0.30240514391553347, 0.13430249537645692, 0.2894590996068156, 0.4832581663989279, 0.16011539985358733, 0.3275953307613343, 0.2930067372057864, 0.09965719933390321, 0.17829660196291008, -0.12515190269999685, 0.0009287629789014873, 0.22840327407596633, 0.10396299816519572, 0.3185853194079814, 0.30761642548595913, 0.15626224912851555, 0.05343763851691811, 0.1335565586600775, 0.05961622513554449, 0.22074943585134305, 0.3655742973362556, 0.14206192113363447, 0.2737744154896256, 0.22840327407596633, 0.2416359259016554, 0.31685766647322133, 0.20126938878852293, 0.07775265026154989, 0.22840327407596633, 0.07811040723581339, 0.2556891959069868, 0.3188911971459635, 0.07021870659389631, 0.22646302953097092, 0.2860096812275518, 0.29770283571505207, 0.39778007444501984, 0.2

In [47]:
E_labels = []
for i in e_scores:
    if i < 0:
        E_labels.append('NEGATIVE ENTAILMENT')
    elif i > 0.3:
        E_labels.append('PERFECT ENTAILMENT')
    else:
        E_labels.append('NEUTRAL ENTAILMENT')

In [48]:
ctrr = 0
for i in e_scores:
    if i < 0:
        ctrr = ctrr + 1
print(ctrr)

7


In [68]:
df_final_E = pd.DataFrame(list(zip(rw_sentence_pairs, aa, e_scores, E_labels)), columns =['sentence pairs', 'entailment scores (raw: e, n, c)', 'entailment_metric [-1:1]', 'entailment_labels'])
df_final_E.to_csv('scores/summary_entailment_scores_of_test_pairs.csv')
df_final_E.head()

,sentence pairs,"entailment scores (raw: e, n, c)",entailment_metric [-1:1],entailment_labels
0,(saudi stock exchange appointed sarah al ceo i...,"[0.24731164388214988, 0.2060030725278753, -0.3...",0.286861,NEUTRAL ENTAILMENT
1,(hollywood actor morgan freeman said would lik...,"[0.2576244015869797, -0.0989217967988879, 0.22...",-0.006846,NEGATIVE ENTAILMENT
2,(uk india business council chairperson said br...,"[0.6017685480474289, -0.023536088484584904, -0...",0.393107,PERFECT ENTAILMENT
3,(indian railways wednesday increased lower ber...,"[0.3295480524571151, -0.062226016292205415, -0...",0.140763,NEUTRAL ENTAILMENT
4,(two pilots arrested saturday glasgow scotland...,"[0.8139928166435216, -0.1578474866966397, -0.1...",0.359204,PERFECT ENTAILMENT


In [53]:
maxi_1 = torch.max(prediction_1, dim=0)
mini_1 = torch.min(prediction_1, dim=0)
# print(maxi[0][0])
denom1_1 = float(maxi_1[0][0].to('cpu')) - float(mini_1[0][0].to('cpu'))
denom2_1 = float(maxi_1[0][1].to('cpu')) - float(mini_1[0][1].to('cpu'))
denom3_1 = float(maxi_1[0][2].to('cpu')) - float(mini_1[0][2].to('cpu'))
denom_1 = ( denom1_1 + denom2_1 + denom3_1 ) / 3
print(maxi_1, mini_1)
print(denom_1)

maxi_2 = torch.max(prediction_2, dim=0)
mini_2 = torch.min(prediction_2, dim=0)
# print(maxi[0][0])
denom1_2 = float(maxi_2[0][0].to('cpu')) - float(mini_2[0][0].to('cpu'))
denom2_2 = float(maxi_2[0][1].to('cpu')) - float(mini_2[0][1].to('cpu'))
denom3_2 = float(maxi_2[0][2].to('cpu')) - float(mini_2[0][2].to('cpu'))
denom_2 = ( denom1 + denom2 + denom3 ) / 3
print(maxi_2, mini_2)
print(denom_2)

torch.return_types.max(
values=tensor([0.5129, 2.3959, 3.6791], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([50, 71, 68], device='cuda:0')) torch.return_types.min(
values=tensor([-1.3504, -0.2311, -1.9773], device='cuda:0', grad_fn=<MinBackward0>),
indices=tensor([81,  4, 79], device='cuda:0'))
3.3822434147198996
torch.return_types.max(
values=tensor([1.0785, 1.8554, 2.2000], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([79, 14, 63], device='cuda:0')) torch.return_types.min(
values=tensor([-0.8711, -0.3274, -1.9962], device='cuda:0', grad_fn=<MinBackward0>),
indices=tensor([97, 90, 14], device='cuda:0'))
3.3822434147198996


In [54]:
p1 = []
for i in prediction_1:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_1)
    p1.append(p)
print(p1)

p2 = []
for i in prediction_2:
    p = []
    for j in i:
        p.append(float(j.to('cpu'))/denom_2)
    p2.append(p)
print(p2)

[[-0.2990544215095602, 0.4881491836951262, 0.022660937920790564], [-0.28391490918508966, 0.2250585363777605, 0.35558400138726787], [-0.07759776791794418, 0.19139587867597568, 0.03372683117700098], [-0.0971400229748619, 0.3038765167111119, -0.06608506584925791], [0.01161434164531932, -0.06833753456311004, 0.40112494642297564], [-0.18152484741800964, 0.22178990980555957, 0.17548836414089944], [0.04027053427112939, 0.24995083088187758, -0.21501400852127647], [-0.20147433044837107, 0.263297873980847, 0.16267915168592903], [-0.15714449564491817, 0.019605681765413507, 0.6232873320138349], [-0.10259322637205864, 0.18790333663594647, 0.08391901823776153], [-0.11271638722220315, 0.29199036960238556, -0.020801605538168854], [-0.33548793179454256, 0.17960105298660134, 0.8140990496692242], [-0.11588377073107389, 0.07100822788600518, 0.3417445240117736], [-0.2216626026011214, 0.48261209570585784, -0.10834445797909152], [0.06263465114480586, 0.2722904608562508, -0.280197411931238], [-0.0174716890236

In [55]:
me1_scores = []
for i in p1:
    me1_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me1_scores)

me2_scores = []
for i in p2:
    me2_scores.append((i[0] + i[1]*0.5 - i[2])/2.5)
print(me2_scores)

[-0.031056307033115054, -0.21078785695339092, -0.006250663902782927, 0.04835332049198078, -0.16947174882368454, -0.09844730266245172, 0.15210398329333788, -0.09300181805755064, -0.3082515947104185, -0.037024230516738775, 0.021632161246863395, -0.42391458198818643, -0.16884967231993794, 0.05119516129235961, 0.1915909174016677, -0.020368398453692108, -0.08299865446882093, 0.10736783514045985, 0.12322385757688603, -0.07263403111548561, -0.21611460256575826, 0.04476580734793193, 0.004528197779190671, -0.12737818208166957, -0.25166713078304004, -0.052507965731969274, 0.09644414212834301, -0.08395022471583727, -0.04149549074792013, -0.17233328311810542, 0.008071340203508836, -0.07915431172824376, -0.01766415658081233, 0.15491438851906597, 0.07561636979617778, -0.07043645746556274, -0.18063345731667935, -0.301016647772563, -0.0029531707215634554, 0.09459328147272208, -0.3195420594436883, 0.04520693104078545, -0.04289093913353384, -0.11859317256373489, -0.019705325998992496, -0.094381839432261

In [57]:
mutual_scores= []
for i, j in zip(me1_scores, me2_scores):
    mutual_scores.append( (i + j) / 2 )

In [78]:
MUTUAL_labels = []
n_c = 0
p_c = 0
c_c = 0
for i in mutual_scores:
    if i <= -0.2:
        c_c += 1
        MUTUAL_labels.append('CONTRADICTION')
    elif i > -0.2 and i < 0:
        n_c += 1
        MUTUAL_labels.append('NEGATIVE ENTAILMENT')
    else:
        p_c += 1
        MUTUAL_labels.append('POSITIVE ENTAILMENT')
        
print("POSITIVE Entailments: ", p_c)
print("NEGATIVE Entailments: ", n_c)
print("Contradictions: ", c_c)

POSITIVE Entailments:  45
NEGATIVE Entailments:  48
Contradictions:  7


In [80]:
df_final_ME = pd.DataFrame(list(zip(rw_sentence_pairs, p1, p2, me1_scores, me2_scores, mutual_scores, MUTUAL_labels)), columns =['sentence pairs', 'e_n_c scores 1', 'e_n_c scores 2', 'entailment_metric_1[-1:1]', 'entailment_metric_2[-1:1]', 'mutuality_score', 'mutual_labels'])
df_final_ME.to_csv('scores/mutual_entailment_scores_of_test_pairs.csv')
df_final_ME.head()

,sentence pairs,e_n_c scores 1,e_n_c scores 2,entailment_metric_1[-1:1],entailment_metric_2[-1:1],mutuality_score,mutual_labels
0,(saudi stock exchange appointed sarah al ceo i...,"[-0.2990544215095602, 0.4881491836951262, 0.02...","[-0.019847235466663993, 0.025029230373259707, ...",-0.031056,-0.085248,-0.058152,NEGATIVE ENTAILMENT
1,(hollywood actor morgan freeman said would lik...,"[-0.28391490918508966, 0.2250585363777605, 0.3...","[0.13433713857849158, -0.016025939841530373, 0...",-0.210788,0.043042,-0.083873,NEGATIVE ENTAILMENT
2,(uk india business council chairperson said br...,"[-0.07759776791794418, 0.19139587867597568, 0....","[0.16972836518417, -0.015848945114540502, -0.0...",-0.006251,0.092454,0.043101,POSITIVE ENTAILMENT
3,(indian railways wednesday increased lower ber...,"[-0.0971400229748619, 0.3038765167111119, -0.0...","[0.1831366123142875, -0.04230500878247594, 0.0...",0.048353,0.063010,0.055682,POSITIVE ENTAILMENT
4,(two pilots arrested saturday glasgow scotland...,"[0.01161434164531932, -0.06833753456311004, 0....","[0.08318766266774678, 0.01859184130703473, 0.0...",-0.169472,0.028958,-0.070257,NEGATIVE ENTAILMENT
